In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-AveragePredicate") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-AveragePredicate").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/25 19:25:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/25 19:25:01 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
readall = spark.read.option("header",True) \
  .csv("s3a://test-out/wdcfix/**")

csall=readall.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csall.createOrReplaceTempView("CSET")
csall.show(truncate=150)

23/04/25 19:25:03 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                                                                                  pset|       count|
+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|                                                                                     <schema.org/item> <schema.org/position> isa:<schema.org/ListItem>|6.82461002E8|
|                                                                                          <schema.org/itemListElement> isa:<schema.org/BreadcrumbList>|4.56168963E8|
|                                                                   <schema.org/item> <schema.org/name> <schema.org/position> isa:<schema.org/ListItem>| 3.9303227E8|
|   

In [30]:
from pyspark.sql.functions import col
def findISA(topn = None):
    if(topn is not None):
        param = f"limit {topn}"
    else:
        param = ""
    
    sets = csall.withColumn("pset", split(csall["pset"], " "))
    
    distinct_predicate = sets.selectExpr("explode(pset) as predicate", "count") \
                    .groupBy("predicate").agg(f.sum(f.col("count")).alias("count"))  \
                    .createOrReplaceTempView("newsets")
    distinct_predicate=spark.sql(f"select * from newsets where predicate like '%isa:%' and count > 1 order by count desc {param}")    
    return distinct_predicate

In [31]:
distinct_predicate = findISA(10000)
#distinct_predicate.show()
distinct_predicate_list = distinct_predicate.select('predicate').rdd.flatMap(lambda x: x).collect()
print(len(distinct_predicate_list))

10000


In [28]:
def calculate_average(pred):
    # Danger injection SQL
    pred = pred.replace("'", "\\'")
    sets = spark.sql(f"SELECT pset, count FROM CSET WHERE pset LIKE '%{pred}%'")
    sets = sets.withColumn("pset", split(sets["pset"], " "))
    
    if sets.count() <= 1:
        #print(f"{pred}: error")
        return -1
    
    count_sum = sets.agg({"count": "sum"}).collect()[0][0]
    count_used = sets.selectExpr("sum(size(pset) * count) as count_used").collect()[0][0]

    average = count_used / count_sum
    #print(f"{pred}: {average}")
    return average

In [33]:
#%%time
from pyspark.sql import Row
rows = []
for pred in distinct_predicate_list:
    rows.append(Row(type=pred, average=float(calculate_average(pred)) ))

df = spark.createDataFrame(rows)

In [34]:
df.write.option("header",True) \
            .mode("overwrite") \
            .csv(f"s3a://test-out/average/top10000")

In [41]:
readtest = spark.read.option("header",True) \
  .csv("s3a://test-out/average/top10000/**")
readtest.show(truncate=150)

+-------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|                                                                                                                                             type|average|
+-------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|                                    isa:<streetdir.com/DE/Berlin/Tempelhof-Schoneberg/Schoneberg/Strassen/Passauer-Strasse/Blumenlaeden/ListItem>|   -1.0|
|                                                                                                                          isa:<schema.org/Soicau>|   -1.0|
|isa:<directoriodecalles.org/AR/Cordoba/Departamento-Marcos-Juarez/Cities/Marcos-Juarez/Streets/Boulevard-Roque-Saenz-Pena/Tecnologia-Tv/ListItem>|   -1.0|
|                                                   isa:<streetd

In [48]:
readtestsort = readtest.sort(readtest.average.desc())
readtestsort.show(30, truncate=150)

+---------------------------------------------+-----------------+
|                                         type|          average|
+---------------------------------------------+-----------------+
|                 isa:<schema.org/sportsEvent>| 9.99938040083935|
|        isa:<schema.org/veterinary%20surgeon>|9.987878787878788|
|                 isa:<schema.orgHVACBusiness>|9.985119047619047|
|                isa:<schema.org/ExercisePlan>|9.984782608695653|
|                  isa:<schema.org/oncologist>|9.976689976689977|
|          isa:<schema.org/Website%20Designer>|9.973134328358208|
|       isa:<schema.org/LocalBusiness/massage>|9.970588235294118|
|             isa:<schema.org/endocrinologist>|9.969696969696969|
|                 isa:<schema.org/TruckDealer>|9.964520367936926|
|                      isa:<schema.org/Winery>|9.946354705274043|
|           isa:<schema.org/cardiac%20surgeon>|9.942857142857143|
|                  isa:<schema.org/expositive>|9.926767676767676|
|         

In [ ]:
withouterrors = readtestsort.filter(readtestsort.average != -1)
print(withouterrors.count())
#withouterrors.show(100, truncate=150)